In [ ]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

from pathlib import Path
from livelossplot import PlotLossesKeras

In [ ]:
# создание каталогов под классы
path='C:/Users/gimaevra94/Documents/emotion_detector/data'

base_dir=Path(path).parent/'data'
train_dir=base_dir/'train'
test_dir=base_dir/'test'
val_dir=base_dir/'val'

anger_dir=train_dir/'anger'
disgust_dir=train_dir/'disgust'
fear_dir=train_dir/'fear'
happy_dir=train_dir/'happy'
neutral_dir=train_dir/'neutral'
sad_dir=train_dir/'sad'
surprise_dir=train_dir/'surprise'

anger_dir=test_dir/'anger'
disgust_dir=test_dir/'disgust'
fear_dir=test_dir/'fear'
happy_dir=test_dir/'happy'
neutral_dir=test_dir/'neutral'
sad_dir=test_dir/'sad'
surprise_dir=test_dir/'surprise'

In [ ]:
train = image_dataset_from_directory(directory=train_dir,
                                     label_mode='int',
                                     color_mode='grayscale',
                                     image_size=(64, 64),
                                     crop_to_aspect_ratio=True,
                                     shuffle=True)

In [ ]:
test = image_dataset_from_directory(directory=test_dir,
                                    label_mode='int',
                                    color_mode='grayscale',
                                    image_size=(64, 64),
                                    crop_to_aspect_ratio=True,
                                    shuffle=True)

In [ ]:
# я следовал архитектуре u-net
inp=Input((64,64,1))

# вход нормализуется
norm=Normalization()(inp)

# понижение размерности признакового пространсва
# свертки со страйдом вместо макспулинга
conv_0=Conv2D(8,3,padding='same',activation='relu')(norm)
conv_str_0=Conv2D(1,3,2,padding='same',activation= 'relu')(conv_0)
conv_1=Conv2D(16,3,padding='same',activation='relu')(conv_str_0)
conv_str_1=Conv2D(1,3,2,padding='same',activation= 'relu')(conv_1)
conv_2=Conv2D(32,3,padding='same',activation='relu')(conv_str_1)
conv_str_2=Conv2D(1,3,2,padding='same',activation= 'relu')(conv_2)

# дилатационные свертки
dila_0=Conv2D(64,3,padding='same',activation='relu',dilation_rate=6)(conv_str_2)
dila_1=Conv2D(64,3,padding='same',activation='relu',dilation_rate=12)(conv_str_2)
dila_2=Conv2D(64,3,padding='same',activation='relu',dilation_rate=18)(conv_str_2)

# склейка и перемешивание дилатационных сверток
concat_0=Concatenate()([dila_0, dila_1, dila_2])
conv_1x1_0=Conv2D(64,3,padding='same',activation='relu')(concat_0)

# повышение размерности признакового пространсва и проброс инфы с более ранних слоев
trans_0=Conv2DTranspose(32,3,2,padding='same',activation='relu')(conv_1x1_0)
concat_1=Concatenate()([conv_2, trans_0])
trans_1=Conv2DTranspose(16,3,2,padding='same',activation='relu')(concat_1)
concat_2=Concatenate()([conv_1, trans_1])
trans_2=Conv2DTranspose(8,3,2,padding='same',activation='relu')(concat_2)
concat_3=Concatenate()([conv_0, trans_2])

drop=Dropout(0.0001)(concat_3)
flat=Flatten()(drop)
out=Dense(7,activation='softmax')(flat)

In [ ]:
model=Model(inputs=inp,outputs=out)

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics='accuracy')

In [ ]:
# коллбек ловит val_loss на <=15%, останавливает сеть и сохраняет в файл
class MyCallback(Callback):
    def on_epoch_end(self,epoch,logs=None): 
        val_loss=logs["val_loss"]
        if val_loss<=0.1500:
            self.model.stop_training=True
            self.model.save('model.keras')

In [ ]:
history=model.fit(x=train,
                    epochs=200,
                    validation_data=test,
                    # PlotLossesKeras() рисует график после каждой эпохи
                    callbacks=[PlotLossesKeras(),MyCallback()]
                   )